## SRJ

## Alternating Adapt and HEA

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)




def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                print('Exc. dealing right now is', ash_excitation[i])
                print('The params that are going in', params[i])
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                print('Single Exc. dealing right now is', ash_excitation[i])
                print('Single exc params that are going in', params[i])
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                #print('Print, if this is activated - HF state')
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        #print('The current status of ash_excitation is', ash_excitation)
        if j == 0:
            params = qml.numpy.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
            print('Parameters are', params)
        else:
            params = np.append(params, 0.0) 
            print('Parameters before optimization are', params)


        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = e_th, options = {'disp': True, 'maxiter': 1e8, 'xtol':1e-8, 'ftol':1e-8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)
        print('Updated params are', params)
        print('Updated excitation are', ash_excitation)

        # New state generation - With the updated parameters
        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        # Append the new state to the states list
        states.append(ostate)

    return energies, params, ash_excitation, gs_state, H, qubits


In [2]:
energies, params, ash_excitation, gs_state,H, qubits = adaptvqe(adapt_it = 5, e_th=1e-12)
print('Energies:', energies)


Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.3325688880668922
The adapt iteration now is 0
The expectation value of a⁺(0) a(4) is 9.258183414351251e-08


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(0) a(6) is 0.0
The expectation value of a⁺(1) a(5) is 9.258183412269583e-08


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(1) a(7) is 0.0
The expectation value of a⁺(2) a(4) is 0.0
The expectation value of a⁺(2) a(6) is 1.1698969991369434e-07


...

The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.007417026063753336


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 1.0043469706006808e-18


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 4.680225857723068e-19
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 1.5602654958868367e-06
Highest gradient excitation is [1, 5]
Parameters before optimization are [-0.82882163 -1.28813042 -1.00807026 -1.16384539  0.        ]
Current parameters: [-0.82881996 -1.28813207 -1.00807108 -1.16383509  0.28091908]
Current cost: -1.85042660802438

Current parameters: [-0.83224736 -1.28907894 -1.01501112 -1.16482949  0.28091908]
Current cost: -1.8504351594739696

Current parameters: [-0.83036895 -1.28925139 -1.0155013  -1.1654085   0.28091908]
Current cost: -1.8504359188795858

Current parameters: [-0.83013813 -1.28924726 -1.01541767 -1.16552613  0.28091908]
Current cost: -1.8504359311257046

Optimization terminated successfully.
         Current function value: -1.850436
         Iterations: 4
         Function evaluations: 538
Final updated parameters: [-0.83013813 -1.28924726 -1.01541767 -1.16552613  0.28091908]
Final cost: -1.850435931125

In [5]:
dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
@qml.qnode(dev_meas)
def measure(gs_state):
    qml.StatePrep(gs_state, wires=range(qubits))
    return qml.counts()

counts = measure(gs_state)
print('Counts:', counts)

Counts: {'00001111': 224, '00111100': 132, '01001011': 6, '01100110': 126, '01111000': 4, '10000111': 4, '10011001': 107, '10110100': 5, '11000011': 119, '11110000': 273}


In [3]:
paramss = np.array([0.1] * (2 * qubits), requires_grad=True)
dev3 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev3)
def k2(paramss):
    qml.StatePrep(gs_state, wires=range(qubits))
    
    # First RY layer (params[0-7])
    for i in range(qubits):
        qml.RY(paramss[i], wires=i)
    
    # CNOT gates
    for i in range(qubits - 1):
        qml.CNOT(wires=[i, i+1])
    
    # Second RY layer (params[8-15])
    for i in range(qubits):
        qml.RY(paramss[i + qubits], wires=i)
    
    return qml.expval(H)

print('Params before optimizeation', paramss)

print(qml.draw(k2, max_length=200)(paramss))


optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

for n in range(1000):
    paramss, energy = optimizer.step_and_cost(k2, paramss)
    print('The 1st set params after calculation :', paramss)
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))
print(qml.draw(k2, max_length=200)(paramss))

Params before optimizeation [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
0: ─╭|Ψ⟩──RY(0.10)─╭●──RY(0.10)─────────────────────────────────────────────────────────────┤ ╭<𝓗>
1: ─├|Ψ⟩──RY(0.10)─╰X─╭●─────────RY(0.10)───────────────────────────────────────────────────┤ ├<𝓗>
2: ─├|Ψ⟩──RY(0.10)────╰X────────╭●─────────RY(0.10)─────────────────────────────────────────┤ ├<𝓗>
3: ─├|Ψ⟩──RY(0.10)──────────────╰X────────╭●─────────RY(0.10)───────────────────────────────┤ ├<𝓗>
4: ─├|Ψ⟩──RY(0.10)────────────────────────╰X────────╭●─────────RY(0.10)─────────────────────┤ ├<𝓗>
5: ─├|Ψ⟩──RY(0.10)──────────────────────────────────╰X────────╭●─────────RY(0.10)───────────┤ ├<𝓗>
6: ─├|Ψ⟩──RY(0.10)────────────────────────────────────────────╰X────────╭●─────────RY(0.10)─┤ ├<𝓗>
7: ─╰|Ψ⟩──RY(0.10)──────────────────────────────────────────────────────╰X─────────RY(0.10)─┤ ╰<𝓗>
The 1st set params after calculation : [0.10745693 0.0931414  0.10382028 0.09836348 0.10924233 0.11056329
 0.10889

In [4]:
dev3 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev3)
def ostatehea(paramss):
    qml.StatePrep(gs_state, wires=range(qubits))
    
    # First RY layer (params[0-7])
    for i in range(qubits):
        qml.RY(paramss[i], wires=i)
    
    # CNOT gates
    for i in range(qubits - 1):
        qml.CNOT(wires=[i, i+1])
    
    # Second RY layer (params[8-15])
    for i in range(qubits):
        qml.RY(paramss[i + qubits], wires=i)
    
    return qml.state()
fstate = ostatehea(paramss)
print(qml.draw(ostatehea, max_length=200)(paramss))

0: ─╭|Ψ⟩──RY(0.00)──╭●──RY(-0.00)─────────────────────────────────────────────────────────────────┤  State
1: ─├|Ψ⟩──RY(-0.00)─╰X─╭●──────────RY(0.00)───────────────────────────────────────────────────────┤  State
2: ─├|Ψ⟩──RY(-0.00)────╰X─────────╭●─────────RY(-0.00)────────────────────────────────────────────┤  State
3: ─├|Ψ⟩──RY(-0.00)───────────────╰X────────╭●──────────RY(-0.00)─────────────────────────────────┤  State
4: ─├|Ψ⟩──RY(0.08)──────────────────────────╰X─────────╭●──────────RY(3.14)───────────────────────┤  State
5: ─├|Ψ⟩──RY(0.00)─────────────────────────────────────╰X─────────╭●─────────RY(-0.00)────────────┤  State
6: ─├|Ψ⟩──RY(1.57)────────────────────────────────────────────────╰X────────╭●──────────RY(1.57)──┤  State
7: ─╰|Ψ⟩──RY(1.62)──────────────────────────────────────────────────────────╰X──────────RY(-1.57)─┤  State


In [35]:
import pennylane as qml

# Initialize device with 1000 shots
dev4 = qml.device("lightning.qubit", wires=qubits, shots=1000)

@qml.qnode(dev4)
def measurement(fstate):
    qml.StatePrep(fstate, wires=range(qubits))
    return qml.counts(all_outcomes = False)  # Samples all qubits

counts = measurement(fstate)
print(counts)


{'10001011': 284, '10101001': 2, '10101010': 714}


In [46]:
fstate = np.array([1,0,1,0,1,0,0,1])

In [47]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)




def adaptvqe_updated(fstate, adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for GS state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - GS state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, fstate, H):
        #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        qml.BasisState(fstate, wires=range(qubits))
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(fstate, ash_excitation, params):
        #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        qml.BasisState(fstate, wires=range(qubits))
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    def cost(params):
        energy = ash(params, ash_excitation, fstate, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [fstate]
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else fstate  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, fstate): # If the current state is the HF state
                print('f state is activated')
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            #print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        print('The current status of ash_excitation is', ash_excitation)
        if j == 0:
            params = qml.numpy.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
            print('Parameters are', params)
        else:
            params = np.append(params, 0.0) 
            print('Parameters before optimization are', params)


        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = e_th, options = {'disp': True, 'maxiter': 1e8, 'xtol':1e-8, 'ftol':1e-8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)
        print('Updated params are', params)
        print('Updated excitation are', ash_excitation)

        # New state generation - With the updated parameters
        ostate = new_state(fstate, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(fstate,ash_excitation,params))
        gs1_state = ostate
        # Append the new state to the states list
        states.append(ostate)

    return energies, params, ash_excitation, gs1_state, H, qubits


In [48]:
energies, params, ash_excitation, gs1_state,H, qubits = adaptvqe_updated(fstate, adapt_it = 10, e_th=1e-12)
print('Energies:', energies)

Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.3325688880668924
The adapt iteration now is 0
f state is activated
f state is activated
f state is activated
f state is activated
f state is activated


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

f state is activated
f state is activated
f state is activated
f state is activated


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


f state is activated


...

The highest operator value is 1.0456240246751704e-08 for operator a⁺(0) a(4)
Highest gradient excitation is [0, 4]
The current status of ash_excitation is [[2, 6], [3, 7], [0, 3, 5, 6], [1, 2, 4, 7], [0, 4], [3, 7], [0, 4], [0, 4]]
Parameters before optimization are [-1.57079626e+00 -1.57079656e+00 -3.14159452e+00 -3.14159226e+00
  3.14159268e+00 -1.09928677e-06  5.51081686e-09  0.00000000e+00]
Current parameters: [-1.57079626e+00 -1.57079627e+00 -3.14159419e+00 -3.14159226e+00
  3.14159268e+00 -9.96616028e-07  5.70002031e-09 -3.27268392e-08]
Current cost: -1.8833260023673337

Optimization terminated successfully.
         Current function value: -1.883326
         Iterations: 1
         Function evaluations: 207
Final updated parameters: [-1.57079626e+00 -1.57079627e+00 -3.14159419e+00 -3.14159226e+00
  3.14159268e+00 -9.96616028e-07  5.70002031e-09 -3.27268392e-08]
Final cost: -1.883326002367334
Updated params are [-1.57079626e+00 -1.57079627e+00 -3.14159419e+00 -3.14159226e+00
  3.1

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

The highest operator value is 1.9002038168825584e-08 for operator a⁺(3) a(7)
Highest gradient excitation is [3, 7]
The current status of ash_excitation is [[2, 6], [3, 7], [0, 3, 5, 6], [1, 2, 4, 7], [0, 4], [3, 7], [0, 4], [0, 4], [3, 7]]
Parameters before optimization are [-1.57079626e+00 -1.57079627e+00 -3.14159419e+00 -3.14159226e+00
  3.14159268e+00 -9.96616028e-07  5.70002031e-09 -3.27268392e-08
  0.00000000e+00]
Current parameters: [-1.57079630e+00 -1.57079627e+00 -3.14159383e+00 -3.14159158e+00
  3.14159269e+00 -1.04911628e-06  2.52904265e-09 -3.27135266e-08
  6.04521065e-09]
Current cost: -1.883326002367349

Optimization terminated successfully.
         Current function value: -1.883326
         Iterations: 1
         Function evaluations: 230
Final updated parameters: [-1.57079630e+00 -1.57079627e+00 -3.14159383e+00 -3.14159158e+00
  3.14159269e+00 -1.04911628e-06  2.52904265e-09 -3.27135266e-08
  6.04521065e-09]
Final cost: -1.883326002367349
Updated params are [-1.57079630

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

The highest operator value is 2.566589259995036e-08 for operator a⁺(2) a(6)
Highest gradient excitation is [2, 6]
The current status of ash_excitation is [[2, 6], [3, 7], [0, 3, 5, 6], [1, 2, 4, 7], [0, 4], [3, 7], [0, 4], [0, 4], [3, 7], [2, 6]]
Parameters before optimization are [-1.57079630e+00 -1.57079627e+00 -3.14159383e+00 -3.14159158e+00
  3.14159269e+00 -1.04911628e-06  2.52904265e-09 -3.27135266e-08
  6.04521065e-09  0.00000000e+00]
Current parameters: [-1.57079630e+00 -1.57079627e+00 -3.14159383e+00 -3.14159158e+00
  3.14159269e+00 -1.07775052e-06  1.74706436e-09 -3.31041818e-08
  5.41154287e-09 -2.67155449e-10]
Current cost: -1.883326002367349

Optimization terminated successfully.
         Current function value: -1.883326
         Iterations: 1
         Function evaluations: 247
Final updated parameters: [-1.57079630e+00 -1.57079627e+00 -3.14159383e+00 -3.14159158e+00
  3.14159269e+00 -1.07775052e-06  1.74706436e-09 -3.31041818e-08
  5.41154287e-09 -2.67155449e-10]
Final c

In [49]:
-1.883326 - -1.88512490

0.0017989000000000477

In [15]:
print(gs1_state)

[ 0.00000000e+00+9.58308288e-21j  1.08462569e-18+4.59988025e-19j
  6.77626358e-20-4.31238771e-20j  9.62964972e-35+1.16584206e-18j
 -4.96450701e-15+1.62630326e-19j  7.66646708e-19-1.34163174e-19j
  1.00622381e-19-7.66646708e-20j -9.07378023e-14+0.00000000e+00j
 -1.30850096e-13+4.33680869e-18j  2.69859641e-17+4.33709412e-19j
  3.06658683e-19+2.83898859e-19j  2.83442714e-12+0.00000000e+00j
 -9.62964972e-35+1.34163174e-18j -3.90313047e-17+2.45326947e-18j
  1.32801214e-16-5.64251977e-17j  6.77249582e-07-5.63787591e-18j
  2.71050543e-20-6.70815871e-20j  2.71474060e-19+7.66646708e-19j
 -1.35525272e-20+1.43746235e-20j -4.81482486e-35-1.16584205e-18j
 -8.62477547e-20+5.74985031e-20j -2.49160180e-16+9.99498878e-19j
 -9.26298490e-15-2.71050543e-19j -1.08708105e-19-6.13317367e-19j
 -5.55219922e-19+3.67990420e-18j  2.45326947e-18-1.64797671e-17j
  3.51737510e-16-7.42000862e-19j -3.90311329e-18-9.81307787e-18j
  0.00000000e+00-1.76089170e-19j -1.30104261e-18-5.15186588e-17j
  1.55684656e-18+6.133173

In [24]:
import pennylane as qml

# Initialize device with 1000 shots
dev4 = qml.device("lightning.qubit", wires=qubits, shots=10000)

@qml.qnode(dev4)
def measurement(gs1_state):
    qml.StatePrep(gs1_state, wires=range(qubits))
    return qml.counts()  # Samples all qubits

samples = measurement(gs1_state)
print(samples)



{'00101110': 864, '10000111': 2, '10001000': 4, '10001011': 940, '10100110': 11, '10101001': 4, '10101010': 7210, '10110100': 2, '10111000': 963}


In [10]:
dev = qml.device("default.qubit", wires=2, shots=4)

@qml.qnode(dev)
def circuit(x):
    qml.RX(x, wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.sample()

circuit(0.1)

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1]])

In [8]:
from collections import defaultdict

# Convert samples to binary strings (e.g., "00", "11")
binary_states = [''.join(map(str, sample)) for sample in samples]

# Count occurrences
counts = defaultdict(int)
for state in binary_states:
    counts[state] += 1

# Find the most frequent state
most_common = max(counts, key=lambda k: counts[k])
print(f"Predominant state: |{most_common}⟩ (Count: {counts[most_common]})")

#Predominant state: |10101010⟩ (Count: 709)


Predominant state: |10101010⟩ (Count: 69)


## HEA ansatz now

paramss1 = np.array([0.1] * (2 * qubits), requires_grad=True)
dev3 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev3)
def k3(paramss1):
    qml.StatePrep(gs1_state, wires=range(qubits))
    
    # First RY layer (params[0-7])
    for i in range(qubits):
        qml.RY(paramss1[i], wires=i)
    
    # CNOT gates
    for i in range(qubits - 1):
        qml.CNOT(wires=[i, i+1])
    
    # Second RY layer (params[8-15])
    for i in range(qubits):
        qml.RY(paramss1[i + qubits], wires=i)
    
    return qml.expval(H)

print('Params before optimizeation', paramss1)

print(qml.draw(k3, max_length=200)(paramss1))


optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

for n in range(1000):
    paramss1, energy = optimizer.step_and_cost(k3, paramss1)
    print('The 1st set params after calculation :', paramss1)
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))
print(qml.draw(k3, max_length=200)(paramss1))